In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

c:\Users\Ken\anaconda3\envs\fyp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
import torch
import torchvision
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
import os
import json
from PIL import Image


class LvisDataset(Dataset):
    def __init__(self,json_path):
        if json_path and os.path.exists(json_path):
            with open(json_path) as user_file:
                file_contents = user_file.read()
            parsed_json = json.loads(file_contents)
            
            # print(parsed_json.keys())
            # dict_keys(['info', 'categories', 'annotations', 'images', 'licenses'])
            # separate them into three different list
            images = parsed_json['images']
            annotations = parsed_json['annotations']
            categories = parsed_json['categories']
            # print(len(images),len(annotations),len(categories))
            # new_images = []
            # new_annotations = []
            # new_categories = []
            images_d = {}
            bbox_d = {}
            
        
            # for i in range(len(images)):
            #     if images[i]['id'] not in images_d.keys():
            #         image_name = images[i]['coco_url'].split('/')[-1]
            #         path_prefix ="./train/"
            #         image_path = os.path.join(path_prefix, image_name)
            #         image = Image.open(image_path)
            #         to_tensor = torchvision.transforms.ToTensor()
            #         image = to_tensor(image)
            #         images_d[images[i]['id']] = [image]
            #         break
            #     else:
            #         # images_dic[images[i]['id']].append(images[i])
            #         print("Error: duplicate image id")
            
            # for i in range(len(categories)):
            #     if categories[i]['id'] not in categories_dic.keys():
            #         categories_dic[categories[i]['id']] = [categories[i]]
            #     else:
            #         categories_dic[categories[i]['id']].append(categories[i])      
            
            for i in range(len(annotations)):
                img_id = annotations[i]['image_id']
                cat_id = annotations[i]['category_id']
                one_hot = torch.zeros(len(categories))
                one_hot[cat_id] = 1
                box_N_label = torch.cat((torch.tensor(annotations[i]['bbox']), one_hot))
                if img_id not in bbox_d.keys():
                    bbox_d[img_id] = box_N_label
                else:
                    bbox_d[img_id] = torch.stack((bbox_d[img_id], box_N_label), dim = 0)
                break
                # new_images.append(annotations[i])
                # new_annotations.append(images_dic[annotations[i]['image_id']][0])
                # new_categories.append(categories_dic[annotations[i]['category_id']][0])
            
            self.images = images_d
            self.bbox = bbox_d
            # self.annotations = new_annotations
            # self.categories =  new_categories
            # self.n_samples = len(new_images)  
            
                

    
    def __getitem__(self, index):
        return self.images[index] , self.bbox[index]

    def __len__(self):
        return len(self.images)
    
if __name__ == '__main__':
    a = LvisDataset(json_path="lvis_v1_train.json")
    

In [10]:
lin = nn.Linear(512, 1203, bias=False)
clip = np.load("clip_synonym_prompt.npy")
lin_params = lin.state_dict()
lin_params["weight"] = torch.from_numpy(clip)
lin(torch.randn((3,512))).shape
# for k in lin_params:
    # print(k)

torch.Size([3, 1203])

# Items needed
1.  A fking LVIS data loader
2.  A video processing framework -> UI 
3.  matplotlib codesss
4.  VILD inference stage 

In [ ]:
def try_gpu():
    return "cuda:0" if torch.cuda.is_available() else "cpu"
def train(model, lr, num_epochs, train_iter, test_iter, pretrain_path=None,):
    device =try_gpu()
    trainer=torch.optim.AdamW(params = model.parameters(), lr=lr)
    model = model.to(device)
    if pretrain_path is not None:
        model.load_state_dict(torch.load(pretrain_path))
    model.train()
    min_loss = 999
    # plt.figure(figsize=(10,10))
    X_rec=[[],[]]
    Y_rec=[[],[]]
    for epoch in num_epochs:
        proposal_losses, frcnn_losses, feat_distill_loss, model_loss, train_l = train_one_epoch(model, train_iter, trainer)
        if (epoch + 1) % 5  or epoch == 0:
            train_l = np.array(train_l).mean()
            test_l = eval_acc(model, test_iter, device = device, batch_size = batch_size)
            print(f"Epoch {epoch+1} Train Loss : {train_l}, Test Loss : {test_l}")
            X_rec,Y_rec=add(fig,axe,epoch+1,[train_l,test_l],X_rec,Y_rec)
            
            # torch.save(net.state_dict(), f"weight/resnet_{epoch+1}.pt")
        if(test_l<min_loss):
            torch.save(net.state_dict(), "weight/resnet_best.pt")
            test_l=min_loss
    return 0
def train_one_epoch(model, train_iter, trainer):
    '''
    loss function needed: rpn_score, rpn_box, frcnn_class, frcnn_box, feat_distill
    rpn scr loss: rpn_scr (from proposal loss) -> "loss_objectness"
    rpn box loss: rpn_box (from proposal loss) -> "loss_rpn_box_reg"
    frcnn cls loss: cls_output (from frcnn loss) -> "loss_classifier"
    frcnn box loss: box_outputs (from frcnn loss) -> "loss_box_reg"
    feat distill loss: distill_feat, compute the roi and only use those with roi greater than 0, multiples 0.5(feat_distill_weight)
        
    '''
    # load the visual features from data loader
    # 
    train_l = []
    for x,y in train_iter:
        class_output, bbox_output, distill_feat, proposal_losses, frcnn_losses, feat_distill_loss, model_loss = model(x)
        model_loss.backward()
        trainer.step()
        train_l.append(model_loss.item())
    return proposal_losses, frcnn_losses, feat_distill_loss, model_loss, train_l